In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow
import tensorflow as tf
import h5py
import math
import keras
from PIL import Image
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model, to_categorical
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

/opt/conda/lib/python3.10/site-packages/keras/backend.py:452: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn(


## Creating some imp function for loading dataset

In [2]:
def load_dataset():
    train_dataset = h5py.File('../input/signs-detection-dataset/Signs_Data_Training.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels

    test_dataset = h5py.File('../input/signs-detection-dataset/Signs_Data_Testing.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels

    classes = np.array(test_dataset["list_classes"][:]) # the list of classes
    
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

In [5]:
def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)].T
    return Y

In [6]:
mbnet_v2 = keras.applications.MobileNetV2(
    weights="imagenet",
    include_top=False,
    input_shape=(64, 64, 3)
)

# Stop from being trainable
mbnet_v2.trainable = False

9406464/9406464 [==============================] - 0s 0us/step


## Load  dataset

In [5]:
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

# Normalize image vectors
X_train = X_train_orig/255.
X_test = X_test_orig/255.

# Convert training and test labels to one hot matrices
Y_train = convert_to_one_hot(Y_train_orig, 6).T
Y_test = convert_to_one_hot(Y_test_orig, 6).T

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

number of training examples = 1080
number of test examples = 120
X_train shape: (1080, 64, 64, 3)
Y_train shape: (1080, 6)
X_test shape: (120, 64, 64, 3)
Y_test shape: (120, 6)


In [6]:
Y_train

array([[0., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       ...,
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.]])

## Compiling the model

In [11]:
inputs = keras.Input(shape=(64, 64, 3))

# Get the layer
x = mbnet_v2(inputs, training = False)

# Stack layers further
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(6, activation="softmax")(x)

# Combine the model
model = Model(inputs=inputs, outputs=x)

# Summary
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, 2, 2, 1280)       2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 6)                 7686      
                                                                 
Total params: 2,265,670
Trainable params: 7,686
Non-trainable

In [12]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

## Fit the model

In [33]:
# Num epochs
epochs = 50

# Train model
history = model.fit(
    X_train,
    Y_train,
    epochs=epochs,
)

Epoch 1/50
34/34 [==============================] - 0s 10ms/step - loss: 0.0274 - accuracy: 0.9972
Epoch 2/50
34/34 [==============================] - 0s 8ms/step - loss: 0.0299 - accuracy: 0.9963
Epoch 3/50
34/34 [==============================] - 0s 7ms/step - loss: 0.0246 - accuracy: 0.9981
Epoch 4/50
34/34 [==============================] - 0s 7ms/step - loss: 0.0248 - accuracy: 0.9963
Epoch 5/50
34/34 [==============================] - 0s 7ms/step - loss: 0.0212 - accuracy: 0.9981
Epoch 6/50
34/34 [==============================] - 0s 7ms/step - loss: 0.0222 - accuracy: 0.9991
Epoch 7/50
34/34 [==============================] - 0s 7ms/step - loss: 0.0227 - accuracy: 0.9981
Epoch 8/50
34/34 [==============================] - 0s 7ms/step - loss: 0.0294 - accuracy: 0.9954
Epoch 9/50
34/34 [==============================] - 0s 7ms/step - loss: 0.0193 - accuracy: 1.0000
Epoch 10/50
34/34 [==============================] - 0s 7ms/step - loss: 0.0242 - accuracy: 0.9981
Epoch 11/50
34/34 

In [36]:
val_loss = history.history["loss"]
val_accuracy = history.history["accuracy"]

In [14]:
preds = model.evaluate(X_test, Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

4/4 [==============================] - 1s 12ms/step - loss: 0.1744 - accuracy: 0.9417
Loss = 0.1743667870759964
Test Accuracy = 0.9416666626930237


# TinyML

In [15]:
converter = tensorflow.lite.TFLiteConverter.from_keras_model(model)

In [16]:
tflite_model = converter.convert()

In [17]:
with open("model.tflite", "wb") as f:
    f.write(tflite_model)

In [19]:
# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path="/kaggle/working/model.tflite")
interpreter.allocate_tensors()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [20]:
img = "/kaggle/input/index-finger/500px-Index_finger_%3D_to_attention.JPG"

In [21]:
from PIL import Image

image = Image.open(img)
image_array = np.array(image)

In [22]:
image_resized = image.resize((64, 64))
image_array_resized = np.array(image_resized)

In [23]:
final_array = np.expand_dims(image_array_resized, axis=0)

In [24]:
final_array = final_array.astype('float32')
final_array /= 255.0

In [25]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Prepare input data
input_data = final_array  # Provide input data as a numpy array
interpreter.set_tensor(input_details[0]['index'], input_data)

# Run inference
interpreter.invoke()

# Get predictions
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

[[6.5309519e-07 9.9938583e-01 4.9517833e-04 1.0827330e-04 3.6981906e-08
  9.9804229e-06]]


In [26]:
print(np.argmax(output_data[0]))

1


# Quantization

In [27]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]  # Enable optimizations
tflite_model = converter.convert()

# Save the model
with open("model_quantized.tflite", "wb") as f:
    f.write(tflite_model)


In [28]:
interpreter = tf.lite.Interpreter(model_path="/kaggle/working/model_quantized.tflite")
interpreter.allocate_tensors()


In [29]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Prepare input data
input_data = final_array  # Provide input data as a numpy array
interpreter.set_tensor(input_details[0]['index'], input_data)

# Run inference
interpreter.invoke()

# Get predictions
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

[[8.0640547e-08 9.9970537e-01 2.5538108e-04 3.7099711e-05 2.9541324e-08
  1.9770844e-06]]
